Basic To advanced PySpark query

# DATA READING  

In [0]:
# Read Data from CSV:
df_csv = spark.read.csv('/FileStore/tables/employees.csv',inferSchema=True, header=True) 
display(df_csv)  # or df_csv.display()

# inferSchema means auto identify the data type in CSV / Json file.

# same as json 
#df_json = spark.read.json('/FileStore/tables/employees.json',inferSchema=True, header=True)

employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
100,Steven,King,SKING,515.123.4567,2087-06-17,AD_PRES,25000,null,null,90
101,Neena,Kochhar,NKOCHHAR,515.123.4568,2089-09-21,AD_VP,17000,null,100,90
102,Lex,De Haan,LDEHAAN,515.123.4569,2093-01-13,AD_VP,17000,null,100,90
103,Alexander,Hunold,AHUNOLD,590.423.4567,2090-01-03,IT_PROG,9000,null,102,60
104,Bruce,Ernst,BERNST,590.423.4568,2091-05-21,IT_PROG,6000,null,103,60
105,David,Austin,DAUSTIN,590.423.4569,2097-06-25,IT_PROG,4800,null,103,60
106,Valli,Pataballa,VPATABAL,590.423.4560,2098-02-05,IT_PROG,4800,null,103,60
107,Diana,Lorentz,DLORENTZ,590.423.5567,2099-02-07,IT_PROG,4200,null,103,60
108,Nancy,Greenberg,NGREENBE,515.124.4569,2094-08-17,FI_MGR,12000,null,101,100
109,Daniel,Faviet,DFAVIET,515.124.4169,2094-08-16,FI_ACCOUNT,9000,null,108,100


## Data Reading Utils

In [0]:
# Csv/Json uploaded locatations in Data bricks
dbutils.fs.ls('dbfs:/FileStore/tables/')

Out[6]: [FileInfo(path='dbfs:/FileStore/tables/departments.csv', name='departments.csv', size=687, modificationTime=1738674464000),
 FileInfo(path='dbfs:/FileStore/tables/emp.json', name='emp.json', size=752, modificationTime=1739564468000),
 FileInfo(path='dbfs:/FileStore/tables/employees.csv', name='employees.csv', size=7920, modificationTime=1738674451000)]

**Delete a File in locatation**

In [0]:
#Delete a Single File
dbutils.fs.rm("dbfs:/FileStore/tables/employees-1.csv")


#Delete All Files in the Folder
#dbutils.fs.rm("dbfs:/FileStore/tables/", recurse=True)   #The recurse=True flag ensures all files in the folder are deleted.


Out[5]: True

**Schema** **Definition**

In [0]:

# print the description of table / Scheme
df_csv.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- job_id: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- commission_pct: double (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)



In [0]:
df_csv.sort(['salary'],assending=[1]).display()

employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
132,TJ,Olson,TJOLSON,650.124.8234,2099-04-10,ST_CLERK,2100,null,121,50
128,Steven,Markle,SMARKLE,650.124.1434,2000-03-08,ST_CLERK,2200,null,120,50
136,Hazel,Philtanker,HPHILTAN,650.127.1634,2000-02-06,ST_CLERK,2200,null,122,50
127,James,Landry,JLANDRY,650.124.1334,2099-01-14,ST_CLERK,2400,null,120,50
135,Ki,Gee,KGEE,650.127.1734,2099-12-12,ST_CLERK,2400,null,122,50
119,Karen,Colmenares,KCOLMENA,515.127.4566,2099-08-10,PU_CLERK,2500,null,114,30
131,James,Marlow,JAMRLOW,650.124.7234,2097-02-16,ST_CLERK,2500,null,121,50
140,Joshua,Patel,JPATEL,650.121.1834,2098-04-06,ST_CLERK,2500,null,123,50
144,Peter,Vargas,PVARGAS,650.121.2004,2098-07-09,ST_CLERK,2500,null,124,50
182,Martha,Sullivan,MSULLIVA,650.507.9878,2099-06-21,SH_CLERK,2500,null,120,50


In [0]:
n.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- job_id: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- commission_pct: double (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)



## StructType() Schema

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *